In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("BUILD_DISPLAY_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;

string database_name = String.IsNullOrEmpty(dbname)? "DB_Benchmarks" : dbname;
string table_name = String.Concat(buildname, "_", buildNr);

In [ ]:
string basepath = @"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\strongScaling\IBM_RotCube\";
List<System.Data.DataTable> LoadedTabs = new List<System.Data.DataTable>();
/*
LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_2_2021-9-24.json"));
LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_1_2021-9-24.json"));
LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_2021-9-24.json"));
*/
LoadedTabs.Add(basepath+@"\"+table_name+".json");
var SesTab = LoadedTabs.Pick(0);
LoadedTabs.ForEach(s=>SesTab.Merge(s)); SesTab.Rows.Count()

In [ ]:
SesTab.GetColumnNames().Where(s=>s.Contains("Name"))

In [ ]:
string[] ColumnNames = new string[]{
"NoOfCores",
"TotalRuntime",
"MatrixAssembly_time",
"AggregationBaseInit_time",
"DataIO_time",
"CGProjection_time",
"SayeCompile_time",
"StandardCompile_time",
"AMR_time",
"LoadBal_time",
"SolverInit_time",
"SolverRun_time"};

In [ ]:
int k = 4;

In [ ]:
var ProfileTab = SesTab.ExtractColumns(ColumnNames);
ProfileTab.Clear();
foreach(System.Data.DataRow Row in SesTab.Rows){
    if(Convert.ToInt32(Row["DGdegree:Velocity*"])!=k) continue;
    var newrow = ProfileTab.NewRow();
    //double total = Convert.ToDouble(Row["TotalRuntime"]);
    foreach(string CName in ColumnNames){
        newrow[CName]=Convert.ToDouble(Row[CName]);
    }
    ProfileTab.Rows.Add(newrow);
}

In [ ]:
string Name_Ncores = "NoOfCores";
string Name_DGdegree = "DGdegree:Velocity*";

In [ ]:
// Average of Repititions ... pt1: Group Runs
var RegistryOfRuns = new Dictionary<string,List<System.Data.DataRow>>();
foreach(System.Data.DataRow Row in ProfileTab.Rows){
    string test = "";
    test += "_c"+Convert.ToString(Row[Name_Ncores]);
    List<System.Data.DataRow> Reps;
    if(RegistryOfRuns.TryGetValue(test,out Reps)){
        Reps.Add(Row);
    } else {
        Reps = new List<System.Data.DataRow>();
        Reps.Add(Row);
        RegistryOfRuns.Add(test,Reps);
    }
} RegistryOfRuns.OrderBy(s=>s.Key[0]).OrderBy(s=>s.Key[1]).Select(s=>new object[]{s.Key,s.Value.Count()})

In [ ]:
RegistryOfRuns["_c8"][0][1]

In [ ]:
// Average of Repititions ... pt2: Compute Average
var tmp = ProfileTab.Clone();
tmp.Clear();
foreach(var kvp in RegistryOfRuns){
    var newrow = tmp.NewRow();
    var sample = kvp.Value.First();
    newrow[Name_Ncores]=Convert.ToInt32(sample[Name_Ncores]);
    foreach(System.Data.DataColumn col in tmp.Columns){
        if(col.ColumnName==Name_Ncores) continue;
        double value=0.0;
        foreach(System.Data.DataRow row in kvp.Value){
            try{
                value+=Convert.ToDouble(row[col.ColumnName]);
            } catch (Exception ex){
                Console.WriteLine("err at "+col.ColumnName);
            }
        }
        newrow[col]=value / kvp.Value.Count();
    }
    tmp.Rows.Add(newrow);
}
ProfileTab = tmp;
ProfileTab.Print()

In [ ]:
//ProfileTab.Columns["NoOfCores"].SetOrdinal(0);
System.Data.DataView view = new System.Data.DataView(ProfileTab);
view.Sort = "NoOfCores";
ProfileTab=view.ToTable();
ProfileTab.Print()

In [ ]:
/*
System.Data.DataTable dtNew = new System.Data.DataTable();
var dt=ProfileTab;
dtNew.Columns.Add("times",typeof(String));
//adding columns    
for(int i=0; i<dt.Rows.Count; i++)
{
    dtNew.Columns.Add(Convert.ToString(dt.Rows[i]["NoOfCores"]));
}
//Changing Column Captions: 


//Adding Row Data
for(int k=1; k<dt.Columns.Count; k++)
{
    System.Data.DataRow r = dtNew.NewRow();
    r[0] = dt.Columns[k].ToString(); 
    for(int j=1; j<=dt.Rows.Count; j++)
    r[j] = dt.Rows[j-1][k];  
    dtNew.Rows.Add(r);
}
dtNew.Print()
*/

In [ ]:
string dirpath = Directory.GetCurrentDirectory();
ProfileTab.ToCSVFile(dirpath+@"\profile_k"+k+".dat"); dirpath